In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib

# === 1. Cargar modelo y binarizador
model = tf.keras.models.load_model("models/best_model_3x1024_dropout04.keras")
mlb = joblib.load("models/label_binarizer.joblib")
label_names = mlb.classes_

# === 2. Cargar dataset a predecir
X_pred = pd.read_csv("X_to_predict_ready.csv", index_col='gb_id')

# === 3. Realizar predicción de probabilidades
y_pred_proba = model.predict(X_pred)
y_pred = (y_pred_proba > 0.4).astype(int)

# === 4. Forzar al menos una etiqueta por fila
for i in range(len(y_pred)):
    if y_pred[i].sum() == 0:
        y_pred[i, np.argmax(y_pred_proba[i])] = 1

# === 5. Reconstruir etiquetas predichas
predicted_labels = mlb.inverse_transform(y_pred)
predicted_labels_str = [';'.join(lbls) if lbls else '' for lbls in predicted_labels]

# === 6. Crear DataFrame con predicciones y probabilidades
df_result = pd.DataFrame(y_pred_proba, columns=label_names, index=X_pred.index)
df_result.insert(0, 'predicted_labels', predicted_labels_str)
df_result.insert(0, 'gb_id', X_pred.index)

# === 7. Guardar a CSV
df_result.to_csv("predictions_with_probabilities.csv", index=False)
print("✅ Predicciones y probabilidades guardadas correctamente.")


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
✅ Predicciones y probabilidades guardadas correctamente.
